In [3]:
!pip install openai

In [13]:
import openai
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import time
from sklearn.metrics import accuracy_score


In [2]:
openai.api_key = "sk"


In [3]:
def get_data_lists(data):
    questions = []
    choices = []
    labels = []
    for example in data:
        # print(example.keys())
        questions.append(example['question'])
        choices.append(example['choice_list'])
        labels.append(example['label'])
    return questions, choices, labels

class Brain_Teaser(Dataset):
  def __init__(self, questions, choices, labels, max_len=512):
    self.questions = questions
    self.choices = choices
    self.labels = labels

    self.max_len = max_len
    self.inputs = []
    self.targets = []
    self.question_options_encoded = []

    self.build_questions()
  
  def build_questions(self):
    maxi = 0
    for id in range(len(self.questions)):
      question_options = "Question : " + self.questions[id] + ' ' + "Options 0:" + ' ' + self.choices[id][0] + ' Options 1: ' + self.choices[id][1] + ' Options 2: ' + self.choices[id][2] + ' Options 3: ' + self.choices[id][3]
    #   maxi = max(maxi, len(question_options.split(' ')))
    #   self.question_options_encoded.append(self.tokenizer(question_options, return_tensors="pt", max_length = 205, padding="max_length", truncation=True, add_special_tokens = True))
      self.question_options_encoded.append(question_options)

  def __len__(self):
    return len(self.question_options_encoded)
  
  def __getitem__(self, id):
    return self.question_options_encoded[id],  self.labels[id] 

In [4]:
train_data = np.load("data/SP-train.npy", allow_pickle=True)
test_data = np.load("data/SP_eval_data_for_practice.npy", allow_pickle = True)

train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)
train_questions, train_choices, train_labels = get_data_lists(train_data)
val_questions, val_choices, val_labels = get_data_lists(val_data)
train_dataset = Brain_Teaser(train_questions, train_choices, train_labels)
val_dataset = Brain_Teaser(val_questions, val_choices, val_labels)



In [5]:
print(len(val_dataset))

51


In [8]:
targets = []
predictions = []
for step in range(len(val_dataset)):
    messages = [
        {"role": "system", "content": "You are a MCQ solver based on the question and options, you predict the correct option id from 0,1,2,3"},
    ]
    
    # Delay for 20 seconds
    time.sleep(18)
    
    query = val_dataset[step]
    # print(query)
    messages.append(
        {"role": "user", "content": query[0]},
    )
    chat = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages
    )    
    reply = chat.choices[0].message.content
    targets.append(query[1])
    predictions.append(reply)

    print(f"ChatGPT: {reply}", query[1], step)
    


ChatGPT: Option 0: He is a coach. 0 0
ChatGPT: Option 0: It didn't because roosters don't lay eggs. 0 1
ChatGPT: 1 1 2
ChatGPT: Option 0: The lake was frozen, none of the men got wet despite sinking to the bottom. 1 3
ChatGPT: Option 1: He is bald. 1 4
ChatGPT: Option 1: The horse operates a mill and travels in a circular clockwise direction. The two outside legs will travel a greater distance than the two inside legs. 1 5
ChatGPT: The correct option is: 0: The men were miners. 1 6
ChatGPT: Option 2: Twenty-five dollars 1 7
ChatGPT: Option 3: None of above.

Explanation: Fish do not drown, perish, or swim away like humans. Therefore, all ten fish would still be left in the tank. 3 8
ChatGPT: Option 0: EACH is the name of one of the men. 0 9
ChatGPT: Option 3: None of above 3 10
ChatGPT: Option 1: One girl is polite and returns her apple back. 2 11
ChatGPT: Option 1: Andy has to use the shovel to create a pile of dirt under the window so he can climb up onto it and escape from the cell.

In [14]:
import re

numeric_pred = []
for text in predictions:
    numeric_pred.append(int(re.findall(r'\d+', text)[0]))

accuracy = accuracy_score(targets, numeric_pred)

print("Accuracy:", accuracy)

Accuracy: 0.7254901960784313
